In [ ]:
import cv2
import numpy as np
import os
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from skimage.feature import hog
from skimage import exposure
import matplotlib.pyplot as plt
import joblib

In [ ]:

def load_dataset(dataset_path):
    images = []
    labels = []
    
    for image_name in os.listdir(dataset_path):
        image_path = os.path.join(dataset_path, image_name)
        if image_name.endswith('.jpg') or image_name.endswith('.png'):
            image = cv2.imread(image_path)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) 
            
            label = int(image_name.split('_')[1].split('.')[0])
            
            image_resized = cv2.resize(image, (128, 64)) 
            
            images.append(image_resized)
            labels.append(label)
    
    return np.array(images), np.array(labels)

def extract_hog_features(images):
    hog_features = []
    
    for image in images:
        fd, _ = hog(image, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2),
                    visualize=True, multichannel=False)
        hog_features.append(fd)
    
    return np.array(hog_features)

def train_svm(X_train, y_train):
    model = svm.SVR(kernel='linear') 
    model.fit(X_train, y_train)
    
    return model

def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    
    mae = mean_absolute_error(y_test, y_pred)
    print(f'Mean Absolute Error: {mae:.2f}')
    
    for i in range(5):
        plt.imshow(X_test[i].reshape(64, 128), cmap='gray')  
        plt.title(f'Predicted: {int(y_pred[i])}, True: {int(y_test[i])}')
        plt.show()

if __name__ == '__main__':
    dataset_path = './datasets/train_images'

    images, labels = load_dataset(dataset_path)

    hog_features = extract_hog_features(images)

    X_train, X_test, y_train, y_test = train_test_split(hog_features, labels, test_size=0.2, random_state=42)

    model = train_svm(X_train, y_train)

    evaluate_model(model, X_test, y_test)

    joblib.dump(model, 'svm_hog_model.pkl')
